In [ ]:
import sys 
sys.path.append('..')

In [ ]:
from segment.sam_script import * 
from segment.dino_script import get_dino_results
from segment.utils import load_resize_image
import torch 
from segment.utils import unload_mask
from segment.utils import overlay_mask 


In [ ]:

sam_model = get_sam_model()

In [ ]:
url = "https://i.pinimg.com/originals/f0/72/20/f072205304f54c828b279f9cfdee5548.jpg"


In [ ]:
image = load_resize_image(url, 1024)
text_prompt = 'leg'
print(f"Prompt: {text_prompt}")
image.resize((300,300))

In [ ]:
df = get_dino_results([image], text_prompt)

In [ ]:
df.asdict()

In [ ]:
with torch.no_grad():
    # dino_images = torch.stack([transform_image_dino(image) for image in images])
    sam_images = torch.stack([sam_transform_image(im) for im in [image]])

    # dino_images = dino_images.to(DEVICE)
    sam_images = sam_images.to(DEVICE)

In [ ]:
sam_outputs = sam_detect(sam_model, sam_images, df.boxes)

In [ ]:
output = {}

total = len(sam_outputs[0].get('masks'))

for row in range(0,total):
    mask = sam_outputs[0].get('masks')[row]
    iou = sam_outputs[0].get('iou_predictions')[row]
    low_res = sam_outputs[0].get('low_res_logits')[row]
    
    
    output[row] = {
        "mask": unload_mask(mask),
        "iou": iou.item(),
        "low_res": low_res,
    }

In [ ]:
num = 1
row = output[0]

mask = row.get('masks')[num]
score = row.get('scores')[num]

print(score)
overlay_mask(image, mask).resize((400,400))